In [1]:
from pathlib import Path
import os
from dotenv import load_dotenv
import pandas as pd
import rasterio

In [2]:
load_dotenv()

True

In [3]:
BASE_DIR = Path(os.getenv("BASE_DIR"))

In [4]:
ELEVATION_STRINGS_PATH = os.getenv("ELEVATION_STRINGS_PATH")

In [4]:

# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.width', None)

In [43]:
dem_path = BASE_DIR/ ELEVATION_STRINGS_PATH/ "NT_SRTM_1sec.tif"
 
with rasterio.open(dem_path) as src:
    crs = src.crs
    print("CRS:", crs)
    print("Is geographic:", crs.is_geographic)
    print("Is projected:", crs.is_projected)

CRS: EPSG:4326
Is geographic: True
Is projected: False


In [47]:
import rasterio
from pathlib import Path

dem_dir = BASE_DIR / ELEVATION_STRINGS_PATH / "For DEM/Target"  # sửa lại đường dẫn
tifs = list(dem_dir.glob("*.tif"))

def inspect_raster(fp):
    with rasterio.open(fp) as ds:
        info = {
            "file": fp.name,
            "crs": ds.crs,
            "width": ds.width,
            "height": ds.height,
            "pixel_size_x": ds.transform.a,
            "pixel_size_y": ds.transform.e,
            "origin_x": ds.transform.c,
            "origin_y": ds.transform.f,
            "bounds": ds.bounds
        }
    return info

infos = [inspect_raster(fp) for fp in tifs]

for i in infos:
    print("\n", "-"*60)
    for k, v in i.items():
        print(f"{k}: {v}")



 ------------------------------------------------------------
file: aspect_30m.tif
crs: EPSG:28352
width: 32429
height: 55667
pixel_size_x: 30.442368760196125
pixel_size_y: -30.442368760196125
origin_x: 500027.79982186557
origin_y: 8787798.3866919
bounds: BoundingBox(left=500027.79982186557, bottom=7093163.044918062, right=1487243.3763462657, top=8787798.3866919)

 ------------------------------------------------------------
file: curv_profile_30m_MGA52.tif
crs: EPSG:28352
width: 32429
height: 55667
pixel_size_x: 30.442368760196125
pixel_size_y: -30.44236876019616
origin_x: 500027.79982187
origin_y: 8787798.3866919
bounds: BoundingBox(left=500027.79982187, bottom=7093163.04491806, right=1487243.37634627, top=8787798.3866919)

 ------------------------------------------------------------
file: NT_DEM_30m_MGA52_bigtif.tif
crs: EPSG:28352
width: 32429
height: 55667
pixel_size_x: 30.442368760196125
pixel_size_y: -30.442368760196125
origin_x: 500027.79982186557
origin_y: 8787798.3866919
bo

In [48]:
import numpy as np

def same_grid(ref, other):
    return (
        ref["crs"] == other["crs"]
        and np.isclose(ref["pixel_size_x"], other["pixel_size_x"])
        and np.isclose(ref["pixel_size_y"], other["pixel_size_y"])
        and np.isclose(ref["origin_x"], other["origin_x"])
        and np.isclose(ref["origin_y"], other["origin_y"])
    )

ref = infos[0]
print(f"Reference grid: {ref['file']}")

for i in infos[1:]:
    ok = same_grid(ref, i)
    print(f"{i['file']}: SAME GRID = {ok}")

Reference grid: aspect_30m.tif
curv_profile_30m_MGA52.tif: SAME GRID = True
NT_DEM_30m_MGA52_bigtif.tif: SAME GRID = True
roughness_30m.tif: SAME GRID = True
slope_30m.tif: SAME GRID = True


In [5]:
folder = Path(BASE_DIR / ELEVATION_STRINGS_PATH / "For DEM" / "Target")  
files = {
    "aspect": folder / "aspect_30m.tif",
    "curv_profile": folder / "curv_profile_30m_MGA52.tif",
    "dem": folder / "NT_DEM_30m_MGA52_bigtif.tif",
    "roughness": folder / "roughness_30m.tif",
    "slope": folder / "slope_30m.tif",
}

def raster_meta(path):
    with rasterio.open(path) as src:
        return {
            "path": str(path.name),
            "crs": src.crs,
            "shape": (src.height, src.width),
            "dtype": src.dtypes[0],
            "nodata": src.nodata,
            "transform": src.transform,
            "res": src.res,
            "bounds": src.bounds,
            "count": src.count,
        }

metas = {k: raster_meta(p) for k, p in files.items()}


for k, m in metas.items():
    print(f"\n--- {k} ---")
    for kk in ["path","crs","shape","dtype","nodata","res","count"]:
        print(f"{kk:10s}: {m[kk]}")
    print(f"bounds    : {m['bounds']}")
    print(f"transform : {m['transform']}")


ref = metas["dem"]

def assert_same(a, b, key):
    if a[key] != b[key]:
        raise ValueError(f"❌ Mismatch {key}: {a['path']} vs {b['path']}\n{a[key]}\n{b[key]}")

for k, m in metas.items():
    assert_same(ref, m, "crs")
    assert_same(ref, m, "shape")
    assert_same(ref, m, "transform")
    assert_same(ref, m, "res")





--- aspect ---
path      : aspect_30m.tif
crs       : EPSG:28352
shape     : (55667, 32429)
dtype     : float32
nodata    : -9999.0
res       : (30.442368760196125, 30.442368760196125)
count     : 1
bounds    : BoundingBox(left=500027.79982186557, bottom=7093163.044918062, right=1487243.3763462657, top=8787798.3866919)
transform : | 30.44, 0.00, 500027.80|
| 0.00,-30.44, 8787798.39|
| 0.00, 0.00, 1.00|

--- curv_profile ---
path      : curv_profile_30m_MGA52.tif
crs       : EPSG:28352
shape     : (55667, 32429)
dtype     : float32
nodata    : nan
res       : (30.442368760196125, 30.44236876019616)
count     : 1
bounds    : BoundingBox(left=500027.79982187, bottom=7093163.04491806, right=1487243.37634627, top=8787798.3866919)
transform : | 30.44, 0.00, 500027.80|
| 0.00,-30.44, 8787798.39|
| 0.00, 0.00, 1.00|

--- dem ---
path      : NT_DEM_30m_MGA52_bigtif.tif
crs       : EPSG:28352
shape     : (55667, 32429)
dtype     : float32
nodata    : -3.4028234663852886e+38
res       : (30.4423

ValueError: ❌ Mismatch transform: NT_DEM_30m_MGA52_bigtif.tif vs curv_profile_30m_MGA52.tif
| 30.44, 0.00, 500027.80|
| 0.00,-30.44, 8787798.39|
| 0.00, 0.00, 1.00|
| 30.44, 0.00, 500027.80|
| 0.00,-30.44, 8787798.39|
| 0.00, 0.00, 1.00|

In [ ]:
out_dir = Path(BASE_DIR / ELEVATION_STRINGS_PATH / "For DEM" / "tabular")
out_dir.mkdir(exist_ok=True)

files = {
    "dem": folder / "NT_DEM_30m_MGA52_bigtif.tif",
    "slope": folder / "slope_30m.tif",
    "aspect": folder / "aspect_30m.tif",
    "roughness": folder / "roughness_30m.tif",
    "curv_profile": folder / "curv_profile_30m_MGA52.tif",
}

srcs = {k: rasterio.open(v) for k, v in files.items()}
ref = srcs["dem"]

chunk_id = 0

for ji, window in ref.block_windows(1):

    arrays = {}
    mask_all = None

    for name, src in srcs.items():
        arr = src.read(1, window=window, masked=True)
        arrays[name] = arr
        mask_all = (~arr.mask) if mask_all is None else (mask_all & ~arr.mask)

    if mask_all.sum() == 0:
        continue

    rows, cols = np.where(mask_all)
    rows_global = rows + window.row_off
    cols_global = cols + window.col_off

    xs, ys = rasterio.transform.xy(
        ref.transform, rows_global, cols_global
    )

    df = pd.DataFrame({
        "x": np.array(xs),
        "y": np.array(ys),
    })

    for name, arr in arrays.items():
        df[name] = arr.data[rows, cols]

    df.to_parquet(out_dir / f"features_chunk_{chunk_id:05d}.parquet")
    chunk_id += 1

    if chunk_id % 10 == 0:
        print(f"Saved {chunk_id} chunks")

# close files
for src in srcs.values():
    src.close()


In [ ]:
out_dir = Path(BASE_DIR / ELEVATION_STRINGS_PATH / "For DEM" / "tabular_500m_bigwindow")
out_dir.mkdir(exist_ok=True)

files = {
    "dem": folder / "NT_DEM_30m_MGA52_bigtif.tif",
    "slope": folder / "slope_30m.tif",
    "aspect": folder / "aspect_30m.tif",
    "roughness": folder / "roughness_30m.tif",
    "curv_profile": folder / "curv_profile_30m_MGA52.tif",
}

srcs = {k: rasterio.open(v) for k, v in files.items()}
ref = srcs["dem"]


In [ ]:
PIXEL = ref.res[0]          # ~30.44 m
TARGET = 500               # m
FACTOR = int(round(TARGET / PIXEL))  # ~16

WINDOW = 2048              
height, width = ref.height, ref.width

print("Downsample factor:", FACTOR)


In [ ]:
def agg(arr):
    d = arr.compressed()
    if d.size == 0:
        return None
    return {
        "mean": float(np.mean(d)),
        "std": float(np.std(d)),
        "p90": float(np.percentile(d, 90)),
    }


In [ ]:
chunk_id = 0

for r0 in range(0, height, WINDOW):
    for c0 in range(0, width, WINDOW):

        window_big = Window(
            col_off=c0,
            row_off=r0,
            width=min(WINDOW, width - c0),
            height=min(WINDOW, height - r0),
        )

       
        arrays = {
            name: src.read(1, window=window_big, masked=True)
            for name, src in srcs.items()
        }

        rows_w, cols_w = window_big.height, window_big.width

        records = []

      
        for r in range(0, rows_w, FACTOR):
            for c in range(0, cols_w, FACTOR):

                feats = {}

                for name, arr in arrays.items():
                    sub = arr[r:r+FACTOR, c:c+FACTOR]

                    if name == "aspect":
                        d = sub.compressed()
                        if d.size == 0:
                            feats = None
                            break
                        rad = np.deg2rad(d)
                        feats["aspect_sin_mean"] = float(np.mean(np.sin(rad)))
                        feats["aspect_cos_mean"] = float(np.mean(np.cos(rad)))
                    else:
                        s = agg(sub)
                        if s is None:
                            feats = None
                            break
                        for k, v in s.items():
                            feats[f"{name}_{k}"] = v

                if feats is None:
                    continue

         
                rr = r0 + r + FACTOR // 2
                cc = c0 + c + FACTOR // 2
                x, y = rasterio.transform.xy(ref.transform, rr, cc)

                feats["x"] = x
                feats["y"] = y
                records.append(feats)

        if not records:
            continue

        df = pd.DataFrame(records)
        df.to_parquet(out_dir / f"cells500m_chunk_{chunk_id:04d}.parquet")
        chunk_id += 1

        if chunk_id % 10 == 0:
            print(f"Saved {chunk_id} chunks")

print("Total chunks:", chunk_id)


In [ ]:
import glob

In [ ]:
files = glob.glob(BASE_DIR / ELEVATION_STRINGS_PATH / "For DEM" / "tabular_500m_bigwindow" / "*.parquet")

df_topo = pd.concat(
    (pd.read_parquet(f) for f in files),
    ignore_index=True
)

df_topo.to_parquet(BASE_DIR / ELEVATION_STRINGS_PATH / "For DEM" / "features_topo_500m.parquet")